In [40]:
import datasets

dataset = datasets.load_dataset("deepmind/narrativeqa")

dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [41]:
from IPython.display import display

df = dataset["validation"].to_pandas()
df.head()

,document,question,answers
0,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHO NORMALLY DELIVERS THE OPENING PR...,"[{'text': 'THE ACTOR WEARING THE BLACK CLOAK',..."
1,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHAT NAME WAS CYNTHIA MORE FAMOUSLY ...,"[{'text': 'THE GODDESS DIANA', 'tokens': ['THE..."
2,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,"{'text': 'WHO DOES ECHO WEEP FOR?', 'tokens': ...","[{'text': 'NARCISSUS', 'tokens': ['NARCISSUS']..."
3,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHAT DOES A DRINK FROM NARCISSUS'S S...,"[{'text': 'FALL IN LOVE WITH THEMSELVES', 'tok..."
4,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'IN WHAT VALLEY DID THE SOLEMN REVELS...,"[{'text': 'GARGAPHIE IN GREECE', 'tokens': ['G..."


In [42]:
df['context'] = df['document'].apply(lambda x: x['text'])
df['question'] = df['question'].apply(lambda x: x['text'])
df.head()

,document,question,answers,context
0,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHO NORMALLY DELIVERS THE OPENING PROLOGUE IN ...,"[{'text': 'THE ACTOR WEARING THE BLACK CLOAK',...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
1,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHAT NAME WAS CYNTHIA MORE FAMOUSLY KNOWN BY?,"[{'text': 'THE GODDESS DIANA', 'tokens': ['THE...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
2,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHO DOES ECHO WEEP FOR?,"[{'text': 'NARCISSUS', 'tokens': ['NARCISSUS']...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
3,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHAT DOES A DRINK FROM NARCISSUS'S SPRING CAUS...,"[{'text': 'FALL IN LOVE WITH THEMSELVES', 'tok...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
4,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,IN WHAT VALLEY DID THE SOLEMN REVELS OF CYNTHI...,"[{'text': 'GARGAPHIE IN GREECE', 'tokens': ['G...",ï»¿The Project Gutenberg EBook of Cynthia's Re...


In [43]:
df = df.rename(columns={'question': 'questions'})
df = df.groupby('context').agg({'questions': list}).reset_index()
df.head()

,context,questions
0,"<html>\n\n<head>\n<title>""Domino,"" by Richard ...",[Who planned the robbery that was being invest...
1,<html>\n<head><title>Airplane Script at IMSDb....,"[What is Ted Striker afraid of?, Why is Ted af..."
2,<html>\n<head><title>All About Steve Script at...,"[What is Mary Horowitz's job?, Who is Mary's b..."
3,<html>\n<head><title>American Psycho Script at...,"[Who is the first man Bateman muders?, Who is ..."
4,"<html>\n<head><title>American, The Script at I...","[What does Jack do for a living?, What is the ..."


In [44]:
# Check for duplicates in context column
duplicate_contexts = df['context'].duplicated().sum()
print(f"Number of duplicate contexts: {duplicate_contexts}")
df.shape

Number of duplicate contexts: 0


(115, 2)

In [45]:
# Filter out html documents - we are only interested in text documents
df = df.assign(html=df['context'].str.contains('<html>'))
df['html'].value_counts()
df = df[~df['html']].reset_index(drop=True)
df = df.drop('html', axis=1)
df.head()

,context,questions
0,ï»¿\n Project Gutenberg's Justice (Second Seri...,"[Who is the senior clerk at James How & Sons?,..."
1,ï»¿\nThe Project Gutenberg EBook of The Myster...,"[When does the story begin?, Where does the st..."
2,ï»¿*********The Project Gutenberg Etext of Cri...,"[Where does this story take place?, Who Socrat..."
3,"ï»¿Project Gutenberg etext, The Deliverance; A...",[Which U.S. state is the setting for the story...
4,"ï»¿Project Gutenberg's Armageddon--2419 A.D., ...",[How did Anthony (Buck Rogers) remain asleep f...


In [46]:
df['context_length'] = df['context'].apply(lambda x: len(x))

In [47]:
# Remove documents that are greather than 800000 characters - llama-stack isn't able to handle them atm
df = df[df['context_length'] <= 800_000]
display(df.shape)

# Get first 50 rows
df = df.iloc[:50]
df.shape

(47, 3)

(47, 3)

In [48]:
import pandas as pd

# Save DataFrame to parquet file
df.to_parquet('data/narrativeqa.parquet')
df = pd.read_parquet('data/narrativeqa.parquet')
df.head()

,context,questions,context_length
0,ï»¿\n Project Gutenberg's Justice (Second Seri...,"[Who is the senior clerk at James How & Sons?,...",152636
1,ï»¿\nThe Project Gutenberg EBook of The Myster...,"[When does the story begin?, Where does the st...",352839
2,ï»¿*********The Project Gutenberg Etext of Cri...,"[Where does this story take place?, Who Socrat...",47260
3,"ï»¿Project Gutenberg etext, The Deliverance; A...",[Which U.S. state is the setting for the story...,749261
4,"ï»¿Project Gutenberg's Armageddon--2419 A.D., ...",[How did Anthony (Buck Rogers) remain asleep f...,176572


In [49]:
df.describe()

,context_length
count,47.000000
mean,282956.829787
std,198586.466477
min,45910.000000
25%,102942.500000
50%,244507.000000
75%,377520.500000
max,785669.000000


In [50]:
from counter import get_and_increment_counter
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{get_and_increment_counter()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_51'

In [51]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]
documents[:5]

[{'document_id': '0',
  'content': 'ï»¿\n Project Gutenberg\'s Justice (Second Series Plays), by John Galsworthy\n\n This eBook is for the use of anyone anywhere at no cost and with\n almost no restrictions whatsoever.  You may copy it, give it away or\n re-use it under the terms of the Project Gutenberg License included\n with this eBook or online at www.gutenberg.net\n\n\n Title: Justice (Second Series Plays)\n\n Author: John Galsworthy\n\n Release Date: September 26, 2004 [EBook #2911]\n\n Language: English\n\n Character set encoding: ASCII\n\n *** START OF THIS PROJECT GUTENBERG EBOOK JUSTICE (SECOND SERIES PLAYS) ***\n\n\n\n\n Produced by David Widger\n\n\n\n\n\nGALSWORTHY PLAYS\n\nSECOND SERIES--NO. 1\n\n\nJUSTICE\n\nBy John Galsworthy\n\n\n\nPERSONS OF THE PLAY\n\n JAMES HOW, solicitor\n WALTER HOW, solicitor\n ROBERT COKESON, their managing clerk\n WILLIAM FALDER, their junior clerk\n SWEEDLE, their office-boy\n WISTER, a detective\n COWLEY, a cashier\n MR. JUSTICE FLOYD, a jud

In [52]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

  0%|                                                                                    | 0/47 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████| 47/47 [13:29<00:00, 17.23s/it]
